# Setup

In [0]:
#@title [RUN : 1] Drive Mount
# [RUN : 1]
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#@title [RUN : 2] Change Directory
# [RUN : 2]
# cd for setup
cd drive/My Drive/badminton

In [0]:
## Create directory to store your model data
## Create directory to store your pre trained models and their related config files
# !mkdir data
# !mkdir models

In [0]:
## Clone tensorflow model scripts
# !git clone https://github.com/tensorflow/models tf-models

In [0]:
## To install COCO API
## [[[  in lastly, used coco_clone from github   ]]]

# !cd tf-models
# !git clone https://github.com/cocodataset/cocoapi.git
# !cd cocoapi/PythonAPI
# !make

In [0]:
# Copy compiled modules
# cp -r pycocotools $PROJECT_DIR/tf-models/research/

In [0]:
# Copy compiled modules
# cp -r pycocotools ../../research/

In [0]:
## Install Protobuf (find release for your system - https://github.com/protocolbuffers/protobuf/releases)
!curl -L https://github.com/protocolbuffers/protobuf/releases/download/v3.9.2/protoc-3.9.2-osx-x86_64.zip -o protobuf.zip 
!unzip ./protobuf.zip
!rm ./protobuf.zip
!protoc object_detection/protos/*.proto --python_out=.


# Setting the Work Environment

In [0]:
#@title [RUN : 3] Change Directory
# [RUN : 3]
# If not the first time, start right away
cd tf-models/research

In [0]:
#@title [RUN : 4] Import models for object_detection
# [RUN : 4]
## Import models for object_detection
!protoc object_detection/protos/*.proto --python_out=.

In [0]:
#@title [RUN : 5] Add Tensorflow Libraries to PYTHONPATH
# [RUN : 5]
## Add Tensorflow Libraries to PYTHONPATH
% set_env PYTHONPATH=/content/drive/My Drive/badminton/tf-models/research:/content/drive/My Drive/badminton/tf-models/research/slim

In [0]:
#@title [RUN : 5-1] Test the Installation 
# [RUN : 5-1]
## Test the installation
!python object_detection/builders/model_builder_test.py

***confirm***

*   to check pythonpath -> echo %pythonpath%
*   When you perform RUN[4], you must do it in the tf-models/research directory.

****

***data folder***
*   images, test/train.record, labelmap.pbtxt in data folder
*   Now, we have all the required files ready in data folder.



In [0]:
#@title Download pre-trained model
## Download pre-trained model ==> in models folder.
!curl -L http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz -o ssd_mobilenet_v2_quantized.tar.gz
!tar -xvf ssd_mobilenet_v2_quantized.tar.gz
!rm ssd_mobilenet_v2_quantized.tar.gz


***file information***

1.  model.ckpt — checkpoints of your model (pre-trained model / partially-trained model) with estimators for further training.
2.  pipeline.config — configs of the model, training data, evaluation data and etc.


## Update model pipeline configurations

1.  model
*   Line 3: num_classes = number of classes in your dataset.
2.  train_config
*   Line 157: fine_tune_checkpoint = MODEL_DIR/training/model.ckpt
3.  train_input_reader
*   Line 162: label_map_path = MODEL_DIR/data/labelmap.pbtxt
*   Line 164: input_path = MODEL_DIR/data/train.record
4.  eval_config
*   Line 168: num_examples = number of test images
5.  eval_input_reader
*   Line 174: label_map_path = MODEL_DIR/data/labelmap.pbtxt
*   Line 178: input_path = MODEL_DIR/data/test.record


# Train

In [0]:
#@title [RUN : 6] model_main.py
# [RUN : 6]
# Learning...
# if ckpt-0: 37:36[click to run] ~ 39:35[activate first step] => about 2 min to start learning (19.12.14)

!python object_detection/model_main.py \
    --pipeline_config_path=/content/drive/My\ Drive/badminton/models/training/pipeline.config \
    --model_dir=/content/drive/My\ Drive/badminton/models/training \
    --num_train_steps=100000 \
    --sample_1_of_n_eval_examples=1 \
    --alsologtostderr

# Tflite Converting

In [0]:
#@title Download cocoapi_clone
## helped metrics per category successfully on github
## cocoapi_clone from some user for us
## Without it, 'export_tflite_ssd_graph.py' doesn't work
!git clone https://github.com/philippschw/cocoapi

In [0]:
#@title [RUN : 7] export_tflite_ssd_graph.py
# [RUN : 7]
### The task to optimize before converting to tflite

#export PROJECT_DIR=<YOUR PROJECT DIRECTORY>
#export MODEL_DIR=$PROJECT_DIR/models/<YOUR SSD MODEL>

#export TF_API_DIR=$PROJECT_DIR/tf-models/research
#export MODEL_CONFIG=$MODEL_DIR/pipeline.config
#export CHECKPOINT_PATH=$MODEL_DIR/model.ckpt-<CHECKPOINT NUMBER>  --> input latest number
#export OUTPUT_DIR=$MODEL_DIR/tmp/tflite 

!python object_detection/export_tflite_ssd_graph.py \
--pipeline_config_path=/content/drive/My\ Drive/badminton/models/training/pipeline.config \
--trained_checkpoint_prefix=/content/drive/My\ Drive/badminton/models/training/model.ckpt-60000 \
--output_directory=/content/drive/My\ Drive/badminton/models/training/tmp/tflite \
--add_postprocessing_op=true


*   This will give us two files (tflite_graph.pb&tflite_graph.pbtxt )
*   in the specified output directory (/tmp/tflite).

In [0]:
#@title [RUN : 8 - END] convert optimized pb to tflite 
## bazel 명령어의 workspace의 parent-directory에 공백이 포함되어있으면 실행되지 않음.
## models 파일을 content에 복사하여 작업 진행.
## 그래도 오류 발생하여 bazel을 빼고 바로 toco로 실행 => 성공

!toco --output_file=/content/drive/My\ Drive/badminton/models/training/tmp/tflite/detect-30000.tflite \
--graph_def_file=/content/drive/My\ Drive/badminton/models/training/tmp/tflite/tflite_graph.pb \
--input_shapes=1,300,300,3 \
--input_arrays=normalized_input_image_tensor \
--output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
--inference_type=QUANTIZED_UINT8 \
--mean_values=128 \
--std_dev_values=128 \
--change_concat_input_ranges=false \
--allow_custom_ops

# Tensorboard

In [0]:
#@title Download ngrok
### To local server

! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip

In [0]:
cd ~

In [0]:
cd ../content/drive/My Drive/badminton/models

In [0]:
LOG_DIR = 'training'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')

! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

# Additional Codes [don't need]

In [0]:
## install bazel
BAZEL_VERSION = '0.20.0'
!wget https://github.com/bazelbuild/bazel/releases/download/{BAZEL_VERSION}/bazel-{BAZEL_VERSION}-installer-linux-x86_64.sh
!chmod +x bazel-{BAZEL_VERSION}-installer-linux-x86_64.sh
!./bazel-{BAZEL_VERSION}-installer-linux-x86_64.sh

In [0]:
cd tensorflow

In [0]:
!bazel run -c opt tensorflow/lite/toco:toco -- \
--input_file=/content/models/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03/tmp/tflite/tflite_graph.pb \
--output_file=/content/models/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03/tmp/tflite/detect.tflite \
--input_shapes=1,300,300,3 \
--input_arrays=normalized_input_image_tensor \
--output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
--inference_type=QUANTIZED_UINT8 \
--mean_values=128 \
--std_values=128 \
--change_concat_input_ranges=false \
--allow_custom_ops

In [0]:
cd ~

In [0]:
# model_main.py 실행할 디렉터리로 이동
cd ../content/drive/My Drive/badminton/tf-models/research